# Submit Nanoribbon Workchain

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.code import Code
from aiida.orm import Code, Computer
from aiida.orm.data.base import Int, Float, Str
from aiida.orm.querybuilder import QueryBuilder
from aiida.work.run import run, async, submit
from aiida.orm.data.structure import StructureData

import nglview
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from IPython.display import display, clear_output, HTML
import nglview

from apps.surfaces.structure_browser import StructureBrowser
from apps.surfaces.nanoribbon.nanoribbonwork import NanoribbonWorkChain

In [ ]:
def on_struct_change(c):
    refresh_structure_view()
    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()

clear_output()
display(ipw.VBox([struct_browser, viewer]))

In [ ]:
def refresh_structure_view():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    node = struct_browser.results.value
    if not node:
        return
    atoms = node.get_ase()
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center_view()

    spin_ups = ",".join([str(i) for i, t in enumerate(atoms.get_tags()) if t==1])
    spin_downs = ",".join([str(i) for i, t in enumerate(atoms.get_tags()) if t==2])

    viewer.add_representation('ball+stick', selection="@"+spin_ups, color='red', aspectRatio=3.0, opacity=0.4)
    viewer.add_representation('ball+stick', selection="@"+spin_downs, color='green', aspectRatio=3.0, opacity=0.4)

In [ ]:
# query AiiDA database for Computers
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': 'quantumespresso.pw'})
qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': 'quantumespresso.pp'})
qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': 'quantumespresso.projwfc'})

all_computers = OrderedDict()
all_computers['Please selecte a computer'] = False
for match in qb.all():
    label = match[0]
    codes = match[1:]
    all_computers[label] = codes

drop_computer = ipw.Dropdown(description="Computer", options=all_computers)
slider_precision = ipw.FloatSlider(description='Precision:', value=1.0, min=0.1, max=3.0, step=0.1)
display(ipw.VBox([drop_computer, slider_precision]))

In [ ]:
def on_submit(b):
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not drop_computer.value:
            print("Please select a computer.")
            return

        pw_code, pp_code, projwfc_code = drop_computer.value
        struct = struct_browser.results.value
        precision = Float(slider_precision.value)
        outputs = submit(NanoribbonWorkChain,
                         pw_code=pw_code,
                         pp_code=pp_code,
                         projwfc_code=projwfc_code,
                         structure=struct,
                         precision=precision)
        print(outputs)

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)